# Agent-based peptide discovery (coding)

In [ ]:
import autogen
import os
import tqdm
import time
import openai
from openai import AsyncOpenAI
from llama_index.core import VectorStoreIndex, SimpleDirectoryReader, ServiceContext
from IPython.display import Markdown, display
os.environ["AUTOGEN_USE_DOCKER"] = "0"
from tqdm import tqdm


In [ ]:
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
from llama_index.embeddings.openai import OpenAIEmbedding

In [ ]:
import os

# Credentials are read from environment variables.
# See the repo root `.env.example` (copy to `.env`, do not commit).

# Optional: Hugging Face token (needed for some gated models).
try:
    from huggingface_hub import login
except Exception:
    login = None

hf_token = os.getenv('HF_TOKEN')
if hf_token and login:
    login(token=hf_token)
elif hf_token and not login:
    print('HF_TOKEN is set but `huggingface_hub` is not installed; skipping login.')
else:
    print('HF_TOKEN not set; skipping Hugging Face login.')

# OpenAI/AutoGen expects OPENAI_API_KEY in the environment.
if not os.getenv('OPENAI_API_KEY'):
    raise RuntimeError('OPENAI_API_KEY is not set. See `.env.example`.')


### Using GPT-4 - Main setup starts here

In [ ]:
device = "cpu"
import transformers
autogen.__version__, openai.__version__, transformers.__version__ #make sure to update both autogen and openai to their newsest versions 
# ('0.2.2', '1.6.0', '4.35.1')

In [ ]:
from llm_config import config_list
import agent_functions as func
import agents
import autogen

In [ ]:
def _reset_agents():
    agents.user_proxy.reset()
    #agents.ragproxyagent.reset()
    agents.code_planner.reset()
    agents.code_assistant.reset()
    agents.ml_coder.reset()
    #agents.reviewer.reset()
    #agents.coder.reset()
    #agents.executor.reset()
    #sequence_retriever.reset()
#manager = autogen.GroupChatManager(groupchat=groupchat, llm_config=llm_config)
_reset_agents()

manager_llm_config = {
    "config_list": config_list,
    "seed": 45,
}

groupchat = autogen.GroupChat(
    agents=[agents.user_proxy, agents.code_planner, agents.code_assistant, agents.ml_coder, #agents.coder, agents.executor  #sequence_retriever,
               ], messages=[], max_round=150, 
)
manager = autogen.GroupChatManager(groupchat=groupchat, llm_config = manager_llm_config, 
                system_message='This agent repeats the following steps: Dynamically selecting a speaker, collecting response, and \
broadcasting the message to the group.')

In [ ]:
# agents.user_proxy.initiate_chat(
#     recipient=manager,
#     message='''
# Can you provide some names of antimicrobial resistant bacteria?
# '''
# )
agents.user_proxy.initiate_chat(
    recipient=manager,
    message='''
Given local files: positive_data_fasta_path = 'train_fungi_AMP.fasta' containing antifungal peptide sequences and
negative_data_fasta_path = 'train_negative_peptide.fasta' containing non-antifungal peptide sequences, write codes that can train a deep learing model to classify antifungal peptides vs. non-antifungal peptides. Finetune ESM2 model for this task. Provide the complete code with necessary imports.
'''
)


In [ ]:
agents.user_proxy.send(
    recipient=manager,
    message='''
Given model trained for AMP classification, write codes for the function that can load the model file and inference this model to classify antifungal peptides vs. non-antifungal peptides. Provide the complete code with necessary imports.
'''
)